<h1><center>Solving the Lights Out Puzzle Using Grover's Algorithm</center></h1>
<h2><center>Course Project</center></h2>
<h3><center>CIE470</center></h3>
<h3><center>Introduction to Quantum Information and Computation</center></h3>

## Problem description: 
The Puzzle consists of an NxN grid of bulbs, where each bulb occupied a cell on the grid, and has its own switch, which when pressed switches the state of the bulb between "on <----> off". The grid starts off in an initial state of on-&-off bulbs, and you need to find the sequence of switch presses that would "turn off all the lights", hence the name, "Lights-Out-Puzzle".  


Consider the initial state in the picture attached in the file, with yellow meaning the coresponding bulb is on, this is the setup we will have to solve for. This corresponds to initial state :[0, 1, 1, 0, 0, 1, 1, 0, 0]

then your goal is to find the new state of the switches (either to leave it in the same initial state, 0, or flip the switch,1)

In [49]:
lights = [0, 1, 1, 0, 0, 1, 1, 0, 0] ## this is our variable holding the initial bulb states

## Steps:

#### A. Register Preparation: we need to setup the quantum and classical registers for the problem, which involves using 3 main quantum-registers and a classical-register.

1. A register to hold the solution space of switch-states, since the solution to the problem is a set of 9 on/off(binary system) switches (we have a 3x3 grid of bulbs), call this register "switch/var".
NB: remember a switch can be on or off (binary, and we can only flip a switch once), we therefore have $2^9$ possible states forming our state-space of possible solutions, a perfect candidate for mapping to a 9-qubit system.
2. A control register which will carry temporary info about the state of the bulbs (bulb-states), and be used to determine if a given switch-state is a solution, call it "bulb/ctrl"
3. An output qubit which will initiate a phase kickback if the control register (representing the bulb-states) reflects that a given switch-state is a solution.
4. A classical register to store the measurements.  

#### B. State initialization for our specific problem:
To initilaize the circuit to map to our specific initial state to the register.

#### C. Construct the problem Oracle:
To construct an oracle that would assign a negative phase to the solution state.  

#### D. Construct the Diffuser
 
#### E. Construct the Grover-Circuit over the appropriate number of iterations

#### F. Run the completed Grover-Circuit & Use the measurement to obtain the solution, which corresponds to the most probable state



## Solution: 

### IMPORTANT:   
#### You need to Place the appropriate code in front of the Question cells, please adjust your code so that it fits this framework, no other solution steps will be accepted, if you understood the first phase it should be very easy to follow this framework no matter how you approach the problem.

### A. Preparing registers before we initiate the states and apply an oracle and a Diffuser:

#### Hint: your register will need 19 Qubits

### Q1: Register preparation :

In [84]:
!pip install qiskit-aer
from qiskit import *
import matplotlib.pyplot as plt
import numpy as np

# importing Qiskit
from qiskit import IBMQ, Aer, assemble, transpile
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.providers.ibmq import least_busy

# import basic plot tools
from qiskit.visualization import plot_histogram

'''
Q1: Register preparation :
................................................................................................  
................................................................................................  
................................................................................................  
................................................................................................  
................................................................................................  

 construct the circuit "qc" given below by combining 4 inputs consisting of:
 
 3 quantum registers (switch, bulb, output) and a classical register (result) which returns an initial quantum circuit


'''
switch = QuantumRegister(9, name='s')
bulb = QuantumRegister(9, name='b')
output = QuantumRegister(1, name='out')
result = ClassicalRegister(9, name='result')

qc = QuantumCircuit(switch, bulb, output, result) ## a circuit combining all the registers


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Iterations:

We can always approximate the number of iterations Grover's algorithm needs to reach a solution given the number of winners and the size of our solution state-space, here we have $2^9$ solutions, from which only one is the winner, so describe how many iterations we will need below:

### Q2: Iterations can be determined by :


Q2: Describe how we determine the number of iterations and set the number of iterations for our case:

from the documentation of Grever's algorithm, we know that we should set \theta_t to pi/2 , given that N = 2^n = 2^9
\theta = arcsin(1/sqrt(N)) = 0.044
\theta_t = (2t + 1) \theta
solve for t

In [51]:


iterations = 17  ## initialize the iterations variable

### B. State initialization for our specific problem:

#### Next we need to define a function which
A.  maps the initial bulb-state to the bulb register. for example ... initiate a [0, 1, 1, 0, 0, 1, 1, 0, 0] state, which would correspond to [off, on, on, off, off, on, on, off, off]

B. initializes the $ \left| S \right\rangle $ state (uniform superposition for the Grover circuit) 

C. Initializes the state of the Output Qubit which makes it suitable for phase kickback... you should know which state that is

### Q3: Circuit initialization for the given lights-out setup :

'''  
Describe your code for this step:  

The code should map the light state to the switch, by using the for loop. Then we make the switches in superposition by applying the H gate so all the solutins could be in superposition. Finally, it initilazes the output register in |-> to get the phase kickback.




'''

In [71]:
## a function "initialize" with takes "lights" (the initial bulb-state) "qc" (our quantum circuit) 
## and qr (a temp variable for our register)

def initialize(lights, qc, qr):
    '''Q3: Circuit initialization'''
    ## initialize:
    #A
    for i in range(9):
        if (lights[i]==1):   
            qc.x(qr[i])  
    
    #B
    qc.h(switch)
            
    ## initialize the state of the output qubit for phase kickback
    
    qc.x(output)
    qc.h(output)

### C. Construct the problem Oracle:

#### The Oracle consists of 2 parts, 

First, we must define how a switch-state from our solution state-space affects a bulb-state.  
For example [111111110] is a switch-state where all switches have been flipped, except the last one. Since we know what switching a flip does we can translate this sequence of switch flips to an effect on a bulb-state.  
Accordingly, the first part of the oracle runs a given switch-state yielding a final bulb-state.

Second we must define what "light-state" constitutes a Winner. By definition, our Winner switch-state is the one which renders `ALL bulb-qubits off`. Therefore, the second part of the oracle needs to extract that piece of information from the "bulbs" register, so that in case of finding a winner it could initiate a flip in the output register. This flipping of the output register will in turn initiate a phase-kickback, and we get a negative sign infront of our winner, which is what the oracle does, and we're done.


### Part 1 of the Oracle:  

#### Q4: We need to define a function for the first part of the oracle below, which takes the prepared quantum circuit, some switch state, and the given initial bulb-state, and finally acts on the bulb-state accordingly.

#### Hint: you will need a little bit more than 30 gates to achieve this.

'''    
describe what your code does here for this step:  

according to the game rules, each switch controls the switch's bulb and the adjacent bulbs. Therefore, switch[0] controls bulbls [0,1,3] and so on for the other bulbs.

In [72]:
def switch_flip(qc,switch,bulb):
    '''
    Q4: We need to define a function for the first part of the oracle which translates what
    flipping every switch does to the bulb-state
    
    
       
    describe what your code does here for this step:  '''
    
    qc.cx(switch[0], [ bulb[0], bulb[1], bulb[3] ])
    qc.cx(switch[1], [ bulb[1], bulb[0], bulb[4], bulb[2] ])
    qc.cx(switch[2], [ bulb[2], bulb[1], bulb[5] ])
    qc.cx(switch[3], [ bulb[3], bulb[0], bulb[4], bulb[6] ])
    qc.cx(switch[4], [ bulb[4], bulb[1], bulb[3], bulb[5], bulb[7] ])
    qc.cx(switch[5], [ bulb[5], bulb[2], bulb[4], bulb[8] ])
    qc.cx(switch[6], [ bulb[6], bulb[3], bulb[7] ])
    qc.cx(switch[7], [ bulb[7], bulb[4], bulb[6], bulb[8] ])
    qc.cx(switch[8], [ bulb[8], bulb[5], bulb[7] ])

    

### Part 2 of the Oracle, then combining steps C, D & E into a function which constructs the second part of the Oracle, the Diffuser & obtain the completed Grover-Circuit over the appropriate number of iterations:

#### Q5a: Complete the for loop below which runs our predetermined iterations of Grover.
-applies the first part of the oracle (by calling a function)  
-applies the second part of the oracle to check for the winner switch-state (Hint: a known gate can do this)  
-uncomputes (you should know what that is)  
-applies an appropriate diffuser  

#### Q5b: finally measure the output
Measure the switch-states from which we can infer the most probable answer.

NB:  you can use the "qc.reverse_bits()" function on a "qc"-quantum circuit object to reverse the order of the bits to avoid Qiskit's annoying convention.

'''    
describe what your code does here for these two steps:  

1- the code identifies the winner state and appiles a X gate to make it all ones instead of all zeros. 
2- Apply the multicontroled toffoli gate to mark the state. 
3- uncompute which means going back to the initail state again. 
4- the diffuser which is H X mct X H to reach the amplitude amplification.

the measurement: I applied the X gate to undo its effect beforehand, however it didn't matter in the result.

'''

In [85]:
initialize(lights, qc, bulb)

for i in range(iterations):
    '''Q5a:'''
    '''First part of Oracle'''
    switch_flip(qc, switch, bulb)  # our classical function
    '''
    Multi-qubit controlled gate
    '''
    qc.x(bulb[:])                ##bcz the winner state is all-zeros, and we need an all-ones state so that the phase kickback can be useful
    qc.mct(bulb[:],output[0])
    
        
    '''
    uncompute oracle
    
    '''
    switch_flip(qc, switch, bulb)
    
    
    # diffuser
    qc.h(switch)
    qc.x(switch)
    qc.h(8)
    qc.mct(switch[0:8], 8)
    qc.h(8)
    qc.x(switch)
    qc.h(switch)



# Measure:
'''
Q5b:
measurement
    
'''
qc.x(output[0])
qc = qc.reverse_bits()
qc.measure(switch,result)


### F. Run the Grover-Circuit and use the measurement to obtain the solution:

In [86]:
## Running the completed Grover's circuit:

sim = Aer.get_backend('qasm_simulator')

job = execute(qc, backend=sim, shots=1000)
result = job.result()
count = result.get_counts()

### Q6: Extract the solution form the results:

'''   
Q6  
describe what your code does here for this final step, how do we extract the solution form the results:  

using python function max, we can get the state with the largest amplitude aka the winner. 


'''

In [87]:
''' Q6:

Code to extract solution

'''
winner = max(count, key=count.get)
print (winner[0:3] + "\n" + winner[3:6] + "\n" + winner[6:9])

111
001
101


### Final Remarks: Wrap up your steps, talk briefly about your code and summarize the results and conclusion:  

To be honest, I am impressed by the way we could transform this problem into the quantum realem. I was totally stuck not knowing what to do, and probably won't be able to do any progress without this notebook :3
However, I guess now I have a sense of what to do and how to approach the problem. First, we need to dentify our classical function to help us identify the winner state. Second, use this classcal function in the oracle and then apply the diffuser which is the difficut part. Finally, apply Grover's algorithm to find the wanted state. 


<h3><center>.........................................................  END .............................................................</center></h3>